In [1]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Загружаем логи
event_acc = EventAccumulator('logs/tsp_50/example_training_wo_rtdl_TSP50_20250630T221523/')
event_acc.Reload()

# Собираем все скаляры в DataFrame
scalar_tags = event_acc.Tags()['scalars']
scalar_dfs = []

for tag in scalar_tags:
    data = event_acc.Scalars(tag)
    df = pd.DataFrame([(s.step, s.value) for s in data], columns=["step", tag])
    scalar_dfs.append(df.set_index("step"))

full_df = pd.concat(scalar_dfs, axis=1).reset_index()

# Создаем виджет для выбора колонок
cols = full_df.columns.tolist()
cols.remove("step")  # step всегда по оси X

select_columns = widgets.SelectMultiple(
    options=cols,
    value=[cols[0]] if cols else [],
    description='Метрики',
    rows=10,
    layout=widgets.Layout(width='50%')
)

def plot_selected_metrics(selected):
    plt.figure(figsize=(12, 6))
    for col in selected:
        plt.plot(full_df["step"], full_df[col], label=col)
    plt.xlabel("Step")
    plt.ylabel("Value")
    plt.title("Выбранные метрики TensorBoard")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Подключаем интерактивный виджет
widgets.interact(plot_selected_metrics, selected=select_columns)


interactive(children=(SelectMultiple(description='Метрики', index=(0,), layout=Layout(width='50%'), options=('…

<function __main__.plot_selected_metrics(selected)>

In [34]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def load_scalars(log_path):
    acc = EventAccumulator(log_path)
    acc.Reload()
    scalars = {}
    for tag in acc.Tags()['scalars']:
        entries = acc.Scalars(tag)
        df = pd.DataFrame([(e.step, e.value) for e in entries], columns=['step', tag])
        scalars[tag] = df.set_index('step')
    return scalars

def build_comparison_interface(
    log_path1,
    log_path2,
    plot_title="Сравнение экспериментов",
    label1="Эксперимент 1",
    label2="Эксперимент 2"
):
    # Загружаем скаляры
    data1 = load_scalars(log_path1)
    data2 = load_scalars(log_path2)

    # Находим общие теги
    common_tags = sorted(set(data1.keys()).intersection(data2.keys()))
    if not common_tags:
        print("Нет общих скалярных метрик между логами.")
        return
    print(set(data1.keys())== common_tags)

    # Виджет выбора метрики
    metric_selector = widgets.Dropdown(
        options=common_tags,
        description="Метрика:"
    )

    # Функция построения графика
    def plot_comparison(tag):
        df1 = data1[tag].copy()
        df2 = data2[tag].copy()
        
        # Ограничиваем до общего числа шагов
        max_common_step = min(df1.index.max(), df2.index.max())
        df1 = df1[df1.index <= max_common_step]
        df2 = df2[df2.index <= max_common_step]
        
        plt.figure(figsize=(10, 5))
        plt.plot(df1.index, df1[tag], label=label1)
        plt.plot(df2.index, df2[tag], label=label2)
        plt.title(f"{plot_title}\nМетрика: {tag}")
        plt.xlabel("Step")
        plt.ylabel("Value")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    display(metric_selector)
    widgets.interact(plot_comparison, tag=metric_selector)


In [36]:
build_comparison_interface('logs/tsp_20/example_training_with_rtdl_TSP20_20250628T204020/', 'logs/tsp_20/example_training_wo_rtdl_TSP20_20250623T224717/',
                           label1 = "with_rtdl", label2 = "wo_rtdl", plot_title = "TSP 20")


False


Dropdown(description='Метрика:', options=('grad/actor', 'grad/critic', 'grad_clipped/actor', 'grad_clipped/cri…

interactive(children=(Dropdown(description='Метрика:', options=('grad/actor', 'grad/critic', 'grad_clipped/act…